# Seleção de variáveis

Já vimos em outro notebook que é possível selecionar variáveis com maior importancia para o nosso modelo utilizando a biblioteca Scikit-Learn.
Nesse estudo vamos utilizar a biblioteca Boruta e se possível implementar outras bibliotecas como Boruta-SHAP e MRMR. Até o momento dessa postagem as ultimas 2 bibliotecas apresentavam erros e impossibilitavam sua utilização.

# 1.Importando bibliotecas e conjunto de dados

In [1]:
import numpy as np
import pandas as pd
from boruta import BorutaPy
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, OneHotEncoder

In [2]:
df = pd.read_csv('data/gastos_saude.csv')

# 2.Pré-Processamento

In [3]:
X = df.drop(columns=('expenses'))
Y = df['expenses']

X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.3, random_state=42
)

In [4]:
numericas = X_train.select_dtypes('number').columns
categoricas = X_train.select_dtypes('object').columns

processador_num_1 = Pipeline([('s_scaler', RobustScaler())])

processador_cat_1 = Pipeline([(
    'ohe', OneHotEncoder(drop = "if_binary")
)])

preprocessador_1 = ColumnTransformer(transformers = [
    ("processador_num_1", processador_num_1, numericas),
    ("processador_cat_1", processador_cat_1, categoricas),
    ], remainder = 'passthrough', verbose_feature_names_out=False)

X_train_preproc = pd.DataFrame(
    preprocessador_1.fit_transform(X_train), 
    columns = preprocessador_1.get_feature_names_out()
)
                                                                
X_test_preproc = pd.DataFrame(
    preprocessador_1.transform(X_test),
    columns = preprocessador_1.get_feature_names_out()
)

# Boruta

![Boruta](https://static.wixstatic.com/media/84810c_af41119d68e7476a85ea54c523f3ef45~mv2.jpg/v1/fill/w_540,h_686,al_c,q_85,usm_0.66_1.00_0.01,enc_auto/84810c_af41119d68e7476a85ea54c523f3ef45~mv2.jpg)

O pacote Boruta é uma ferramenta de seleção de características que ajuda a escolher as melhores variáveis ​​ou atributos para treinar um modelo de aprendizado de máquina. Ele foi projetado para encontrar as características mais relevantes em um conjunto de dados, identificando aquelas que têm um impacto significativo nas previsões do modelo.

Trabalha criando conjuntos de dados "falsos" a partir das características existentes, tornando-as aleatórias. Esses conjuntos de dados falsos são chamados de "sombras".

Depois treina um modelo de aprendizado de máquina, como uma floresta aleatória, usando as características reais e as sombras. Ele avalia o desempenho do modelo usando as características reais e as sombras. As características que melhoram significativamente o desempenho em relação às sombras são consideradas relevantes e são selecionadas.

In [5]:
# Instanciando modelo
rfr = RandomForestRegressor(n_jobs=-1, max_depth=5, 
                            random_state=42)

# Definir metodo de seleção de variável do Boruta
seletor_var = BorutaPy(rfr, n_estimators=9, 
                       verbose=2, random_state=42)

# Encontrar todas as variáveis relevantes
seletor_var.fit(X_train_preproc.values, y_train.values)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	9
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	9
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	9
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	9
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	9
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	9
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	9
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	3
Tentative: 	1
Rejected: 	5
Iteration: 	9 / 100
Confirmed: 	3
Tentative: 	1
Rejected: 	5
Iteration: 	10 / 100
Confirmed: 	3
Tentative: 	1
Rejected: 	5
Iteration: 	11 / 100
Confirmed: 	3
Tentative: 	1
Rejected: 	5
Iteration: 	12 / 100
Confirmed: 	3
Tentative: 	1
Rejected: 	5
Iteration: 	13 / 100
Confirmed: 	3
Tentative: 	1
Rejected: 	5
Iteration: 	14 / 100
Confirmed: 	3
Tentative: 	1
Rejected: 	5
Iteration: 	15 / 100
Confirmed: 	3
Tentative: 	1
Rejected: 	5
Iteration: 	16 / 100
Confirmed: 	3
Tentative: 	1
Rejected: 	5
Iteration: 	17 / 

BorutaPy(estimator=RandomForestRegressor(max_depth=5, n_estimators=9, n_jobs=-1,
                                         random_state=RandomState(MT19937) at 0x16CF50E7440),
         n_estimators=9, random_state=RandomState(MT19937) at 0x16CF50E7440,
         verbose=2)

In [6]:
# Verificando variáveis selecionadas
seletor_var.support_

array([ True,  True, False, False,  True, False, False, False, False])

In [7]:
# Verificando ranking das variáveis selecionadas
seletor_var.ranking_

array([1, 1, 2, 4, 1, 3, 5, 5, 7])

In [8]:
# Extrair nomes das variáveis mais importantes
var_selec = seletor_var.support_
nomes_var_importantes = X_train_preproc.columns[var_selec].tolist()
nomes_var_importantes

['age', 'bmi', 'smoker_yes']

In [9]:
# Método transform() para filtrar as variáveis selecionadas
var_select = seletor_var.transform(X_train_preproc.values)
var_select.shape

(936, 3)

# Boruta-SHAP

# MRMR

# Referencias

- https://github.com/scikit-learn-contrib/boruta_py
- https://www.predictiveresearchsolutions.com/post/data-science-tips-feature-selection-using-boruta-in-python
- https://vitaliset.github.io/boruta/
- https://github.com/Ekeany/Boruta-Shap
- https://github.com/smazzanti/mrmr
- https://www.kaggle.com/code/lucamassaron/tutorial-feature-selection-with-boruta-shap
- https://medium.com/data-hackers/como-selecionar-melhores-vari%C3%A1veis-para-o-seu-modelo-com-boruta-ef7cbfb3fc35